In [11]:
import sys
import os
from torch.utils.data import DataLoader
sys.path.insert(0,'/gpfs3/well/rahimi/users/gra027/JNb/')
concordance = []
brier = []
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import HORIZON.CVmortalityPred_inHF. pytorch_pretrained_bert as Bert
from HORIZON.CVmortalityPred_inHF. pytorch_pretrained_bert import optimizer
import sklearn.metrics as skm
from torch.utils.data.dataset import Dataset
from HORIZON.CVmortalityPred_inHF.ModelPkg.utils import *

sys.path.insert(0, '/gpfs3/well/rahimi/users/gra027/JNb/HORIZON/CVmortalityPred_inHF/ModelPkg/')
from HORIZON.CVmortalityPred_inHF.ModelPkg.BEHRTsurv import *
from HORIZON.CVmortalityPred_inHF.ModelPkg.Data_Deterministic import *
from HORIZON.CVmortalityPred_inHF.ModelPkg.utils import *
import matplotlib as plt
from HORIZON.CVmortalityPred_inHF.ModelPkg.BEHRTSodenXcal import *

from torch import optim as toptimizer

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import torch

from HORIZON.CVmortalityPred_inHF.ModelPkg.TriSched import *

from HORIZON.CVmortalityPred_inHF.ModelPkg.CPHloss import * 


In [12]:

global_params = {
    'batch_size': 64,
    'gradient_accumulation_steps': 1,
    'num_train_epochs': 3,
    'device': 'cuda:0',
    'output_dir': "SavedModels/",
    'save_model': True,
    'max_len_seq': 512,
    'max_age': 110,
    'age_year': False,
    'age_symbol': None,
    'fac': 2.0,
    'maxfac': 2.0,
    'diseaseI': 1,
    'treatments': 2,

}

In [13]:
def collate_fn(batch):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    newb = []
    for x in zip(*batch):
        newb.append(np.array([np.array(elem) for elem in x]))
    newbatch = []
    idx = np.argsort(newb[-3].flatten())
    row = np.arange(len(batch))
    for  i , x in enumerate(newb):
        temp = x[idx]
        newbatch.append(temp)
        
    age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat  = newbatch
    return torch.LongTensor(age_ids), torch.LongTensor(input_ids), torch.LongTensor(posi_ids), torch.LongTensor(segment_ids), \
               torch.LongTensor(attMask), torch.FloatTensor(time2event), torch.LongTensor( label),torch.FloatTensor( labelfloat)
        
    

In [14]:
def trainloop(e, binaryFlag=False):
    if binaryFlag:
        print('binary on...')
        dataset_train = SODENXcal_DataLoader(BertVocab['token2idx'], train, global_params['max_len_seq'], max_age=110, year=False, age_symbol=None, min_visit=5)
    else:
        dataset_train = SODENXcal_DataLoader(BertVocab['token2idx'], train, global_params['max_len_seq'], max_age=110, year=False, age_symbol=None, min_visit=5)

    dl_train = DataLoader(dataset_train, global_params['batch_size'], shuffle=True, collate_fn = collate_fn)


    
    z_means = []
    z_std = []
    model.train()
    model.binary=binaryFlag
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt = 0
    temp_lossxcal = 0
    scale2add = ((global_params['maxfac']- global_params['fac']) )/(len(dl_train) * (5/6))
    for step, batch in enumerate(dl_train):
        cnt += 1

        
        age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        time2event = time2event.to(global_params['device'])
        label = label.to(global_params['device'])
        labelfloat = labelfloat.to(global_params['device'])

        logits , outfull,loss= model(input_ids, age_ids, segment_ids, posi_ids, attMask, label, labelfloat, time2event)

        if global_params['gradient_accumulation_steps'] > 1:
            loss = loss / global_params['gradient_accumulation_steps']
            
            
            
        temp_lossxcal =0

        label_4_dcal = labelfloat.clone()
        if e>=0:
            label_4_dcal  =1.0-label_4_dcal
            tgt = cat_bin_target(time2event.clone(), label_4_dcal,global_params['bin_boundaries'], global_params) 
            xcalloss = compute_xcal(outfull, tgt, global_params)
            loss = loss + (global_params['fac']* xcalloss)

        
        loss.backward()
        temp_lossxcal += xcalloss.detach().item()

        temp_loss += loss.item()
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if step % 100 == 0:
            if BINFLAG:
                prec, a, b = precision(logits, labelfloat)
            else:
                prec = -1
            print("epoch: {}\t| Cnt: {}\t| Loss: {:.5f}\t| Dcal fac: {:.5f}\t| Dcal loss: {:.5f}".format(e, cnt, temp_loss / 100, global_params['fac']  , temp_lossxcal))
            temp_loss = 0
            temp_lossxcal = 0
        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
        optim.zero_grad()
        
        tempfac = global_params['fac']
        global_params['fac']  =tempfac +scale2add
        global_params['fac']  =np.minimum(global_params['fac'],global_params['maxfac'])
    # Save a trained model
    del dl_train
    return -1



def evaluationloop(binaryFlag=False):
    if binaryFlag:
        print('binary on...')

        dataset_test = SODENXcal_DataLoader(BertVocab['token2idx'], test, global_params['max_len_seq'], max_age=110, year=False, age_symbol=None, min_visit=5)
    else:
        dataset_test = SODENXcal_DataLoader(BertVocab['token2idx'], test, global_params['max_len_seq'], max_age=110, year=False, age_symbol=None, min_visit=5)

    dl_test = DataLoader(dataset_test, global_params['batch_size'], shuffle=False , collate_fn = collate_fn)

    model.eval()
    y = []
    y_label = []
    loss_temp = 0
    model.binary=binaryFlag
    y_time = []
    
    for step, batch in enumerate(dl_test):
        model.eval()

        
        age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        time2event = time2event.to(global_params['device'])
        label = label.to(global_params['device'])
        labelfloat = labelfloat.to(global_params['device'])

        with torch.no_grad():
            logits , outfull,loss= model(input_ids, age_ids, segment_ids, posi_ids, attMask, label,labelfloat, time2event, False)

            
        tgt = cat_bin_target(time2event, labelfloat,global_params['bin_boundaries'],global_params) 
        xcalloss = compute_xcal(outfull, tgt,global_params)    
            
        
        logits = logits.cpu()
        label = label.cpu()
        labelfloat = labelfloat.cpu()
        time2event = time2event.cpu()
        if step % 50 == 0:
            print(step)
        y_label.append(labelfloat)
        y.append(logits)
        y_time.append(time2event)
        loss_temp = loss_temp+loss.item() 

    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0)
    y_time = torch.cat(y_time, dim=0)
    
    if BINFLAG:
        tempprc, output, label = precision_test(y, y_label)

        tempauroc, output, label = roc_auc(y, y_label)

    else:
        tempprc, output, label = precision_test(y, y_label)

        tempauroc = cindex(y, y_label, y_time)
    
        
    
    return tempprc, tempauroc, loss_temp
    



In [15]:


def validationloop(binaryFlag=False):
    if binaryFlag:
        print('binary on...')

        dataset_test = SODENXcal_DataLoader(BertVocab['token2idx'], valid, global_params['max_len_seq'], max_age=110, year=False, age_symbol=None, min_visit=5)
    else:
        dataset_test = SODENXcal_DataLoader(BertVocab['token2idx'], valid, global_params['max_len_seq'], max_age=110, year=False, age_symbol=None, min_visit=5)

    dl_test = DataLoader(dataset_test, global_params['batch_size']*4, shuffle=False , collate_fn = None)

    model.eval()
    y = []
    y_label = []
    loss_temp = 0
    model.binary=binaryFlag
    y_time = []
    pats=[]
    
    for step, batch in enumerate(dl_test):
        model.eval()

        
        age_ids, input_ids, posi_ids, segment_ids, attMask, time2event, label, labelfloat = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        time2event = time2event.to(global_params['device'])
        label = label.to(global_params['device'])
        labelfloat = labelfloat.to(global_params['device'])

        with torch.no_grad():
            logits , outfull,loss= model(input_ids, age_ids, segment_ids, posi_ids, attMask, label,labelfloat, time2event, True)

        logits = outfull.cpu()
        label = label.cpu()
        labelfloat = labelfloat.cpu()
        time2event = time2event.cpu()
        if step % 50 == 0:
            print(step)
        y_label.append(labelfloat)
        y.append(logits)
        pats.append(label.squeeze(-1))
        y_time.append(time2event)
        loss_temp = loss_temp+loss.item()

    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0)
    y_time = torch.cat(y_time, dim=0)
    pats = torch.cat(pats)
    

        
    
    return y_label, y, y_time,pats
    



# surv modelling

In [16]:
# pwd
import pandas as pd

In [17]:


BertVocab = {}
token2idx = {'MASK': 4,
  'CLS': 3,
  'SEP': 2,
  'UNK': 1,
  'PAD': 0,
            'disease1':5,
             'disease2':6,
             'disease3':7,
             'disease4':8,
             'disease5':9,
             'disease6':10,
             'medication1':11,
             'medication2':12,
             'medication3':13,
             'medication4':14,
             'medication5':15,
             'medication6':16,
            }
idx2token = {}
for x in token2idx:
    idx2token[token2idx[x]]=x
BertVocab['token2idx']= token2idx
BertVocab['idx2token']= idx2token





YearVocab = {'token2idx': {'PAD': 0,
  '1987': 1,
  '1988': 2,
  '1989': 3,
  '1990': 4,
  '1991': 5,
  '1992': 6,
  '1993': 7,
  '1994': 8,
  '1995': 9,
  '1996': 10,
  '1997': 11,
  '1998': 12,
  '1999': 13,
  '2000': 14,
  '2001': 15,
  '2002': 16,
  '2003': 17,
  '2004': 18,
  '2005': 19,
  '2006': 20,
  '2007': 21,
  '2008': 22,
  '2009': 23,
  '2010': 24,
  '2011': 25,
  '2012': 26,
  '2013': 27,
  '2014': 28,
  '2015': 29,
  'UNK': 30},
 'idx2token': {0: 'PAD',
  1: '1987',
  2: '1988',
  3: '1989',
  4: '1990',
  5: '1991',
  6: '1992',
  7: '1993',
  8: '1994',
  9: '1995',
  10: '1996',
  11: '1997',
  12: '1998',
  13: '1999',
  14: '2000',
  15: '2001',
  16: '2002',
  17: '2003',
  18: '2004',
  19: '2005',
  20: '2006',
  21: '2007',
  22: '2008',
  23: '2009',
  24: '2010',
  25: '2011',
  26: '2012',
  27: '2013',
  28: '2014',
  29: '2015',
  30: 'UNK'}}





In [18]:

BINFLAG = False


data = pd.read_parquet('forDemoTRisk2.parquet')
data['baselineage'] = data.study_entry - data.dob 
data['baselineage'] = data['baselineage'].apply(lambda x : str(int(x.days/30)))



In [19]:


ageVocab, _ = age_vocab(max_age=global_params['max_age'], year=global_params['age_year'],
                        symbol=global_params['age_symbol'])

create_folder(global_params['output_dir'])
model_config = {
    'vocab_size': len(BertVocab['token2idx'].keys()),  # number of disease + symbols for word embedding
    'hidden_size': 150,  # word embedding and seg embedding hidden size
    'seg_vocab_size': 2,  # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()),  # number of vocab for age embedding
    'max_position_embedding': global_params['max_len_seq'],  # maximum number of tokens
    'hidden_dropout_prob': 0.3,  # dropout rate
    'num_hidden_layers': 6,  # number of multi-head attention layers required
    'num_attention_heads': 6,  # number of attention heads
    'attention_probs_dropout_prob': 0.4,  # multi-head attention dropout rate
    'intermediate_size': 108,  # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu',
    'initializer_range': 0.02,  # parameter weight initializer range
    'num_treatment': global_params['treatments'],
    'device': global_params['device'],
    'concat_embeddings' : True,
    'time_nums': 51,
    
    

    
}



In [20]:
optim_config = {
    'lr':8e-5, 
    'warmup_proportion': 0.1,
}

In [21]:
sampletrainpat = data.sample(frac=0.8).patid.values
sampletestpat = data[~data.patid.isin(sampletrainpat)].sample(frac=0.5).patid.values
samplevalidpat = data[~data.patid.isin(list(sampletrainpat) + list(sampletestpat))].patid.values

In [22]:
len(np.unique(list(samplevalidpat) +list(sampletestpat) +list(sampletrainpat) ))

3000

In [23]:
cutiter= 0
train = data[data.patid.isin(sampletrainpat)].reset_index(drop=True)
test = data[data.patid.isin(sampletestpat)].reset_index(drop=True)
valid = data[data.patid.isin(samplevalidpat)].reset_index(drop=True)

print('check these 2 numbers are equal --> ', len(data), len(list(sampletrainpat) + list(sampletestpat) + list(samplevalidpat)))




check these 2 numbers are equal -->  3000 3000


In [24]:
len(train), len(test), len(valid), len(train) +  len(test) + len(valid)

(2400, 300, 300, 3000)

In [25]:
len(train), len(test), len(valid), len(train) +  len(test) + len(valid)

(2400, 300, 300, 3000)

In [26]:
set(train.patid.values).intersection(set(test.patid.values)), set(valid.patid.values).intersection(set(test.patid.values)), set(train.patid.values).intersection(set(valid.patid.values))



(set(), set(), set())

In [27]:
dataset_train = SODENXcal_DataLoader(BertVocab['token2idx'], train, global_params['max_len_seq'], max_age=110, year=False, age_symbol=None, min_visit=5)

dl_train = DataLoader(dataset_train, global_params['batch_size']*10, shuffle=True, collate_fn = collate_fn)


In [28]:
bin_boundaries, mid_points = get_bin_boundaries(train, dl_train)
global_params['bin_boundaries'] = bin_boundaries
global_params['mid_points'] = mid_points
global_params['gamma'] = 10000.0

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49]
bin boundaries [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]


In [33]:


import warnings

warnings.filterwarnings(action='ignore')
from sklearn.model_selection import KFold

aurocbest = []
auprcbest = []



conf = BertConfig(model_config)

model = BEHRT_SODENXcal(conf, num_labels=1)




turn on concat - cehrt structure - embeddings


In [34]:

model = model.to(global_params['device'])
optim = adam_surv(params=list(model.named_parameters()), config=optim_config)
model.binary = BINFLAG


scheduler = toptimizer.lr_scheduler.ExponentialLR(optim, 0.95, last_epoch=-1)
patience = 0


In [35]:
auprcbest =[]
aurocbest = []
best_pre = -1
patience = 0
bestauroc = -1
bestloss = 100000000000
# 
for e in range(70):
    fullCounter = trainloop(e, BINFLAG)

    prc, auroc ,loss_temp = evaluationloop(BINFLAG)
    fullCounter = fullCounter+1

    if loss_temp <= bestloss:
        patience=0
        # Save a trained model
        print("** ** * Saving best fine - tuned model ** ** * ")
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join(global_params['output_dir'],  'outbinfileTRisk2.bin')
        create_folder(global_params['output_dir'])
        if global_params['save_model']:
            torch.save(model_to_save.state_dict(), output_model_file)

        best_pre = prc
        bestauroc = auroc
        bestloss=loss_temp
    else:


        if patience >=1 and patience <2:
            scheduler.step()
            print("LR: ", scheduler.get_lr())
        elif patience>=2:
            print("LR: ", scheduler.get_lr())

            print('quitting... no gains...')
            break
        print(patience)
        patience = patience + 1
#     scheduler.step()
    print('lr: ' , scheduler.get_lr())
    print('loss: {}, prc : {}, auroc : {}'.format(loss_temp, prc, auroc))
print('best ever auauroc: ', best_pre)
auprcbest.append(best_pre)
aurocbest.append(bestauroc)
torch.cuda.empty_cache()

epoch: 0	| Cnt: 1	| Loss: 0.14820	| Dcal fac: 2.00000	| Dcal loss: 0.53910
0
** ** * Saving best fine - tuned model ** ** * 
lr:  [8e-05, 8e-05]
loss: 6.080976665019989, prc : 0.2852404120826138, auroc : 0.9439839034205232


KeyboardInterrupt: 

# surv validation

In [ ]:


import warnings

warnings.filterwarnings(action='ignore')
from sklearn.model_selection import KFold

aurocbest = []
auprcbest = []



conf = BertConfig(model_config)

model = BEHRT_SODENXcal(conf, num_labels=1)




In [ ]:
fullBert = os.path.join(global_params['output_dir'],  'outbinfileTRisk2.bin')

pretrained_dict = torch.load(fullBert, map_location='cpu')
net_dict = model.state_dict()
print(len(net_dict))
# # 1. filter out unnecessary keys
pretrained_dict2 = {k: v for k, v in pretrained_dict.items() if k in net_dict }
# # 2. overwrite entries in the existing state dict
net_dict.update(pretrained_dict2) 
model.load_state_dict(net_dict)

model = model.to(global_params['device'])
optim = adam_surv(params=list(model.named_parameters()), config=optim_config)
model.binary = BINFLAG

scheduler = TriStageLRScheduler(optim,  5,5,50, 0.00001,0.000005,-1 )
patience = 0


In [ ]:
auprcbest =[]
aurocbest = []
best_pre = -1
patience = 0
bestauroc = -1


e,ph, t,pats = validationloop(False)


In [ ]:
# sample save of probabilities etc at timepoint 36
np.savez(global_params['output_dir'] +'out_time36output.npz', time=t.flatten().numpy(), event=e.flatten().numpy(), pred=ph.numpy()[:,36],  partial_hazard=ph.numpy(), pats=valid.patid.values)
